In [40]:
import tensorflow as tf
from helper import compute_metrics_base, compute_metrics, measure_cpu_utilization_and_run
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "x_accel",
    "y_accel",
    "z_accel",
]

# Output classes to learn how to classify
LABELS = [
    "Walking", 
    "Jogging", 
    "Uptairs", 
    "Downstairs", 
    "Sitting", 
    "Standing"
] 

In [50]:
from numpy import genfromtxt
import numpy as np
import pandas as pd
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y = pd.read_csv('../Data/WISDM_y.csv')

print(x.shape, y.shape)

(1098209, 3) (1098208, 1)


In [54]:
x[0:16]

array([[ 0.        ,  1.        ,  2.        ],
       [-0.6946377 , 12.680544  ,  0.50395286],
       [ 5.012288  , 11.264028  ,  0.95342433],
       [ 4.903325  , 10.882658  , -0.08172209],
       [-0.61291564, 18.496431  ,  3.0237172 ],
       [-1.1849703 , 12.108489  ,  7.205164  ],
       [ 1.3756552 , -2.4925237 , -6.510526  ],
       [-0.61291564, 10.56939   ,  5.706926  ],
       [-0.50395286, 13.947236  ,  7.0553403 ],
       [-8.430995  , 11.413852  ,  5.134871  ],
       [ 0.95342433,  1.3756552 ,  1.6480621 ],
       [-8.19945   , 19.57244   ,  2.7240696 ],
       [ 1.4165162 ,  5.7886477 ,  2.982856  ],
       [-1.879608  , -2.982856  , -0.29964766],
       [-6.1291566 ,  6.851035  , -8.158588  ],
       [ 5.829509  , 18.0061    ,  8.539958  ]])

In [53]:
def create_series(x,y,timestep,overlap):

    slide_step = int(timestep*(1-overlap))
    data_num = int((len(x)/slide_step)-1)
  
    dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
    labels = list()

    for i in range(data_num):
        labels.append(y[slide_step*(i+1)-1])
        for j in range(timestep):
            dataset[i,j,:] = x[slide_step*i+j,:]

    return dataset,np.array(labels)


In [56]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y).reshape(-1,1)


ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()
y.shape
y[0:16]

/Users/sandeep/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [59]:
# y_pred.append("5")
dataset,labels = create_series(x,y,16,0.5)
print(dataset.shape)
dataset[0:16]

(137275, 16, 3)


array([[[  0.        ,   1.        ,   2.        ],
        [ -0.6946377 ,  12.680544  ,   0.50395286],
        [  5.012288  ,  11.264028  ,   0.95342433],
        [  4.903325  ,  10.882658  ,  -0.08172209],
        [ -0.61291564,  18.496431  ,   3.0237172 ],
        [ -1.1849703 ,  12.108489  ,   7.205164  ],
        [  1.3756552 ,  -2.4925237 ,  -6.510526  ],
        [ -0.61291564,  10.56939   ,   5.706926  ],
        [ -0.50395286,  13.947236  ,   7.0553403 ],
        [ -8.430995  ,  11.413852  ,   5.134871  ],
        [  0.95342433,   1.3756552 ,   1.6480621 ],
        [ -8.19945   ,  19.57244   ,   2.7240696 ],
        [  1.4165162 ,   5.7886477 ,   2.982856  ],
        [ -1.879608  ,  -2.982856  ,  -0.29964766],
        [ -6.1291566 ,   6.851035  ,  -8.158588  ],
        [  5.829509  ,  18.0061    ,   8.539958  ]],

       [[ -0.50395286,  13.947236  ,   7.0553403 ],
        [ -8.430995  ,  11.413852  ,   5.134871  ],
        [  0.95342433,   1.3756552 ,   1.6480621 ],
        [ 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [24]:
# Input Data 

training_data_count = len(X_train)  # 96092 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 41183 testing series
n_steps = len(X_train[0])  # 16 timesteps per series
n_input = len(X_train[0][0])  # 3 input parameters per timestep
print(n_steps,n_input)

# LSTM Neural Network's internal structure
# n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X_train shape, y_train shape, every X_train's mean, every X_train's standard deviation)")
print(X_train.shape, y_train.shape, np.mean(X_train), np.std(X_train))
print("(X_test shape, y_test shape, every X_test's mean, every X_test's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

16 3
Some useful info to get an insight on dataset's shape and normalisation:
(X_train shape, y_train shape, every X_train's mean, every X_train's standard deviation)
(96092, 16, 3) (96092, 6) 2.779314293876691 6.957102673248058
(X_test shape, y_test shape, every X_test's mean, every X_test's standard deviation)
(41183, 16, 3) (41183, 6) 2.769982662671871 6.952643412201609
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [8]:
model_dir = '/Users/sandeep/Desktop/BUCourses/Project/saved_models/CNN'

In [ ]:
model = models.Sequential()
model.add(layers.Conv1D(256, 3, input_shape=(n_steps, n_input)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(128,3))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(64,3))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(32,3))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(6, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])

model.fit(X_train,y_train,batch_size=64,epochs=300,validation_split=0.1)

model.save(model_dir)

In [61]:
model_dir = '/Users/sandeep/Desktop/BUCourses/Project/saved_models/CNN'
model = tf.keras.models.load_model('/Users/sandeep/Desktop/BUCourses/Project/saved_models/CNN')
average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics_base, model, X_test, y_test, model_dir)

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


1287/1287 [==============================] - 1s 1ms/step
Size of the model: 1788.84 KB
Accuracy on the test set: 55.87%
Elapsed time: 1.46 seconds
Average CPU utilization: 48.10%


#### Dynamic Range Quantization

In [10]:
#Implemented Dynamic Range Quantization

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_model_optimization.sparsity import keras as sparsity

#Load saved model
model = tf.keras.models.load_model('/Users/sandeep/Desktop/BUCourses/Project/saved_models/CNN')


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False

# Now, convert the model
cnn_dynamicQ_tflite_model = converter.convert()


# Save the quantized model to a file
with open('/Users/sandeep/Desktop/BUCourses/Project/saved_models/cnn_dynamicQ_model.tflite', 'wb') as f:
    f.write(cnn_dynamicQ_tflite_model)


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqputkrpv/assets


INFO:tensorflow:Assets written to: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqputkrpv/assets
2023-10-19 14:14:38.683189: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-19 14:14:38.683204: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-19 14:14:38.683526: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqputkrpv
2023-10-19 14:14:38.684217: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-19 14:14:38.684221: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/l4/2y2z7y6s4f39zqdkw_9jv51h0000gn/T/tmpqputkrpv
2023-10-19 14:14:38.685611: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-10-19 14:14:38.685964: I tensorflow/cc/saved_model/load

In [12]:
average_cpu_utilization, elapsed_time, result = measure_cpu_utilization_and_run(
                                                compute_metrics, cnn_dynamicQ_tflite_model, X_test, y_test, input_type='float32'
                                                )

print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Average CPU utilization: {average_cpu_utilization:.2f}%")


Size of TFLite model: 142.74 KB
Accuracy on the test set: 55.87%
Elapsed time: 1.15 seconds
Average CPU utilization: 14.35%
